<center><img src="../logo.png" alt="Header" style="width: 800px;"/></center>

# 颜色HSV  Color HSV

In [ ]:
# 中文开关，默认为英文 Chinese switch. The default value is English
g_ENABLE_CHINESE = False

Name_widgets = {
    'Red': ("Red", "红色"),
    'Green': ("Green", "绿色"),
    'Blue': ("Blue", "蓝色"),
    'Yellow': ("Yellow", "黄色"),
    'Custom': ("Custom", "自定义"),
    'Close_Camera': ("Close_Camera", "关闭摄像头")
}

In [ ]:
# 显示摄像头组件 Camera display widgets
import ipywidgets.widgets as widgets
from IPython.display import display

import cv2
import time
import os
import numpy as np


import threading


In [ ]:
#bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
from SunriseRobotLib import Mipi_Camera

img_width=320
img_height=240

g_camera = Mipi_Camera(img_width, img_height)

if g_camera.isOpened():
    print("Open Camera OK")
else:
    print("Fail To Open Camera")


In [ ]:
Color_HSV = {
    'Red': ([0, 43, 46], [10, 255, 255]),
    'Green': ([35, 43, 46], [77, 255, 255]),
    'Blue': ([100, 43, 46], [124, 255, 255]),
    'Yellow': ([26, 43, 46], [34, 255, 255]),
    'Custom': ([0, 0, 0], [180, 255, 255]),
}

# 默认选择自定义颜色，
# The default is Custom.
color_lower = np.array(Color_HSV["Custom"][0])
color_upper = np.array(Color_HSV["Custom"][1])
g_color_name = "Custom"
g_stop_program = False



# 保存HSV值的文件名称 Name of the file in which the HSV value is saved
HSV_NAME_CUSTOM = "CUSTOM_HSV_VALUE.TXT"
HSV_NAME_RED = "RED_HSV_VALUE.TXT"
HSV_NAME_GREEN = "GREEN_HSV_VALUE.TXT"
HSV_NAME_BLUE = "BLUE_HSV_VALUE.TXT"
HSV_NAME_YELLOW = "YELLOW_HSV_VALUE.TXT"


In [ ]:
# image_widget = widgets.Image(format='jpeg', width=320, height=240)
# 摄像头显示控件  Camera display widgets
DISPLAY_WIDTH = img_width
DISPLAY_HEIGHT = img_height
origin_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)
mask_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)
result_widget = widgets.Image(format='jpeg', width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT)

In [ ]:
# 颜色选择按钮配置  Color selection button configuration
# 红色 Red
Button_Red = widgets.Button(       
    description=Name_widgets['Red'][g_ENABLE_CHINESE],
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='uncheck' )

# 绿色 Green
Button_Green = widgets.Button(     
    description=Name_widgets['Green'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 蓝色 Blue
Button_Blue = widgets.Button(        
    description=Name_widgets['Blue'][g_ENABLE_CHINESE],         
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 黄色 Yellow
Button_Yellow = widgets.Button(        
    description=Name_widgets['Yellow'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

# 自定义 Custom
Custombutton = widgets.Button(        
    description=Name_widgets['Custom'][g_ENABLE_CHINESE],        
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='check' )

# 关闭摄像头 Close_Camera
Button_Close_Camera = widgets.Button(
    value=False,  
    description=Name_widgets['Close_Camera'][g_ENABLE_CHINESE],      
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''     
    tooltip='Description',     
    icon='uncheck' )

In [ ]:
# 清除按键显示状态  Clear button display status
def ALL_Uncheck():
    Button_Red.icon = 'uncheck'
    Button_Green.icon = 'uncheck'
    Button_Blue.icon = 'uncheck'
    Button_Yellow.icon = 'uncheck'
    Custombutton.icon = 'uncheck'

# 按键按下事件处理   Key press event processing
def on_button_clicked(b):
    global color_lower, color_upper
    global g_stop_program
    with output:
        print("Button clicked:", b.description)
    if b.description == Name_widgets['Close_Camera'][g_ENABLE_CHINESE]:
        # 停止线程，释放摄像头  Stop the thread and release the camera
        g_stop_program = True
        ALL_Uncheck()
        time.sleep(.1)
        g_camera.release()
        b.icon = 'uncheck'
    else:
        if b.description == Name_widgets['Red'][g_ENABLE_CHINESE]:
            g_color_name = "Red"
            if b.icon == 'uncheck':
                with output:
                    print("Read Red HSV Value")
                succeed = read_hsv_value(HSV_NAME_RED)
                if not succeed:
                    color_lower = np.array(np.array(Color_HSV["Red"][0]))
                    color_upper = np.array(np.array(Color_HSV["Red"][1]))
                    show_hsv(color_lower, color_upper)
            else:
                with output:
                    print("Succeed to Save Red HSV Value")
                save_hsv_value(HSV_NAME_RED)
            
        elif b.description == Name_widgets['Green'][g_ENABLE_CHINESE]:
            g_color_name = "Green"
            if b.icon == 'uncheck':
                with output:
                    print("Read Green HSV Value")
                succeed = read_hsv_value(HSV_NAME_GREEN)
                if not succeed:
                    color_lower = np.array(np.array(Color_HSV["Green"][0]))
                    color_upper = np.array(np.array(Color_HSV["Green"][1]))
                    show_hsv(color_lower, color_upper)
            else:
                with output:
                    print("Succeed to Save Green HSV Value")
                save_hsv_value(HSV_NAME_GREEN)
        elif b.description == Name_widgets['Blue'][g_ENABLE_CHINESE]:
            g_color_name = "Blue"
            if b.icon == 'uncheck':
                with output:
                    print("Read Blue HSV Value")
                succeed = read_hsv_value(HSV_NAME_BLUE)
                if not succeed:
                    color_lower = np.array(np.array(Color_HSV["Blue"][0]))
                    color_upper = np.array(np.array(Color_HSV["Blue"][1]))
                    show_hsv(color_lower, color_upper)
            else:
                with output:
                    print("Succeed to Save Blue HSV Value")
                save_hsv_value(HSV_NAME_BLUE)
        elif b.description == Name_widgets['Yellow'][g_ENABLE_CHINESE]:
            g_color_name = "Yellow"
            if b.icon == 'uncheck':
                with output:
                    print("Read Yellow HSV Value")
                succeed = read_hsv_value(HSV_NAME_YELLOW)
                if not succeed:
                    color_lower = np.array(np.array(Color_HSV["Yellow"][0]))
                    color_upper = np.array(np.array(Color_HSV["Yellow"][1]))
                    show_hsv(color_lower, color_upper)
            else:
                with output:
                    print("Succeed to Save Yellow HSV Value")
                save_hsv_value(HSV_NAME_YELLOW)
        elif b.description == Name_widgets['Custom'][g_ENABLE_CHINESE]:
            g_color_name = "Custom"
            if b.icon == 'uncheck':
                with output:
                    print("Read Custom HSV Value")
                succeed = read_hsv_value(HSV_NAME_CUSTOM)
                if not succeed:
                    color_lower = np.array(np.array(Color_HSV["Custom"][0]))
                    color_upper = np.array(np.array(Color_HSV["Custom"][1]))
                    show_hsv(color_lower, color_upper)
            else:
                with output:
                    print("Succeed to Save Custom HSV Value")
                save_hsv_value(HSV_NAME_CUSTOM)
        ALL_Uncheck()
        b.icon = 'check'


In [ ]:
# 关联按键事件回调 Button event callbacks
Button_Red.on_click(on_button_clicked)
Button_Green.on_click(on_button_clicked)
Button_Blue.on_click(on_button_clicked)
Button_Yellow.on_click(on_button_clicked)
Custombutton.on_click(on_button_clicked)
Button_Close_Camera.on_click(on_button_clicked)

In [ ]:
# 创建滑动条调整HSV的值 Create sliders to adjust the value of HSV
Hmin = widgets.IntSlider(description='Hmin', min=0, max=180, step=1)
Hmax = widgets.IntSlider(description='Hmax', min=0, max=180, step=1)
Smin = widgets.IntSlider(description='Smin', min=0, max=255, step=1)
Smax = widgets.IntSlider(description='Smax', min=0, max=255, step=1)
Vmin = widgets.IntSlider(description='Vmin', min=0, max=255, step=1)
Vmax = widgets.IntSlider(description='Vmax', min=0, max=255, step=1)
# create a horizontal box container to place the sliders next to eachother
slider_H = widgets.HBox([Hmin,Hmax])
slider_S = widgets.HBox([Smin,Smax])
slider_V = widgets.HBox([Vmin,Vmax])
slider_container = widgets.VBox([slider_H, slider_S, slider_V])


In [ ]:
# 更新滑动块的数值 Update the value of the slider
def show_hsv(color_lower, color_upper):
    color_lower = ' '.join(str(color_lower).split())
    color_upper = ' '.join(str(color_upper).split())
    lower = str(color_lower).lstrip("[ ").rstrip("]").split(" ")
    upper = str(color_upper).lstrip("[ ").rstrip("]").split(" ")
    print(lower)
    print(upper)
    Hmin.value=int(lower[0])
    Smin.value=int(lower[1])
    Vmin.value=int(lower[2])
    Hmax.value=int(upper[0])
    Smax.value=int(upper[1])
    Vmax.value=int(upper[2])

# 读取hsv数值，读取成功更新滑块数值，并返回true，读取失败直接返回false。
# read hsv value. The update slider value is returned true if read succeeds, false if read fails.
def read_hsv_value(file_name):
    global color_lower, color_upper
    base_dir = str(os.getcwd())
    base_dir = base_dir.replace('\\', '/')
    file_path = base_dir + "/" + str(file_name)
    try:
        with open(file_path, mode='r') as f_hsv:
            line = f_hsv.readline().rstrip("\n").replace(' ', '').split(",")
            print("read:", line)
            Hmin.value=int(line[0])
            Smin.value=int(line[1])
            Vmin.value=int(line[2])
            Hmax.value=int(line[3])
            Smax.value=int(line[4])
            Vmax.value=int(line[5])
            return True
    except FileNotFoundError:
        print(str(file_name), "File not Found")
        return False
    return False

# 保存HSV文件 save hsv file
def save_hsv_value(file_name):
    base_dir = str(os.getcwd())
    base_dir = base_dir.replace('\\', '/')
    file_path = base_dir + "/" + str(file_name)
    try:
        with open(file_path, mode='w') as f_hsv:
            save_hsv = "%d, %d, %d, %d, %d, %d" % (int(Hmin.value), int(Smin.value), int(Vmin.value), int(Hmax.value), int(Smax.value), int(Vmax.value))
            f_hsv.write(save_hsv)
            print("write:%d, %d, %d, %d, %d, %d" % (int(Hmin.value), int(Smin.value), int(Vmin.value), int(Hmax.value), int(Smax.value), int(Vmax.value)))
    except:
        print("HSV Value File Open Error")

read_ok = read_hsv_value(HSV_NAME_CUSTOM)
if not read_ok:
    show_hsv(color_lower, color_upper)
# save_hsv_value(HSV_NAME_CUSTOM)

In [ ]:
# 图像处理任务 Image processing tasks
def task_processing():
    global color_lower, color_upper
    global g_stop_program
    t_start = time.time()
    m_fps = 0
    while g_camera.isOpened():
        if g_stop_program:
            break
        color_lower = np.array([int(Hmin.value), int(Smin.value), int(Vmin.value)])
        color_upper = np.array([int(Hmax.value), int(Smax.value), int(Vmax.value)])
        ret, frame = g_camera.read()
        # 根据HSV值处理图像 The image is processed according to the HSV value
        hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)
        res = cv2.bitwise_and(frame, frame, mask=mask)
        
        m_fps = m_fps + 1
        fps = m_fps / (time.time() - t_start)
        if (time.time() - t_start) >= 2:
            m_fps = fps
            t_start = time.time() - 1
        cv2.putText(frame, "FPS " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
        
        # 图像传输给显示组件 The image is transmitted to the display component
        origin_widget.value = bgr8_to_jpeg(frame)
        mask_widget.value = bgr8_to_jpeg(mask)
        result_widget.value = bgr8_to_jpeg(res)
        time.sleep(.02)
        

In [ ]:
# 启动摄像头显示任务  Start the camera display task
thread1 = threading.Thread(target=task_processing)
thread1.setDaemon(True)
thread1.start()

# display(image_widget)
# 创建一个横向的盒子容器，以便将图像小部件相邻放置
# create a horizontal box container to place the image widget next to eachother
image_container = widgets.HBox([origin_widget, mask_widget, result_widget])
button_group = widgets.HBox([Button_Red, Button_Green, Button_Blue, Button_Yellow, Custombutton, Button_Close_Camera])
output = widgets.Output()
box_display = widgets.VBox([image_container, button_group, slider_container, output])
display(box_display)